In [ ]:
import glob
import datetime
import pandas as pd
import json
from pandas.io.json import json_normalize

# https://stackoverflow.com/questions/36587211/easiest-way-to-read-csv-files-with-multiprocessing-in-pandas
# http://python-3.ru/page/multiprocessing
from multiprocessing import Pool

import os

In [ ]:
# Создаю/обнуляю список файлов 
files_full_path_list = list()

# Путь к корневому каталогу файлов
files_path = '/opt/app/data/shared/latest_dump/*/*.json'
# files_path = '/opt/app/data/shared/latest_dump/*/2018*.json'

In [ ]:
# Получаю перечень полных пути файлов в подкаталогах
for file_name in glob.glob(files_path, recursive=True):
    # Добавляю полный путь в список
    files_full_path_list.append(file_name)
    
files_full_path_list.sort()

print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'Количество файлов:', len(files_full_path_list))

In [ ]:
files_full_path_list[:10]

In [ ]:
json_df = pd.DataFrame()

In [ ]:
# Создаю результирующий dataframe из json файлов с помощью multiprocessing

def get_file_data(file):
    data_parsed = json.loads(open(file).read())
    df = json_normalize(data_parsed)
    df.insert(loc=0, column='file', value=file)
    return df

pool = Pool(processes=10)
df_list = pool.map(get_file_data, files_full_path_list)

json_df = pd.concat(df_list, ignore_index=True, sort=True)

In [ ]:
# Сохраняю dataframe в csv

csv_file_name = 'ods_data.csv'
csv_file_dir = './'
csv_file_path = csv_file_dir + csv_file_name

# Проверка существует ли файл. Если существует удаляю
if os.path.exists(csv_file_path):
    os.remove(csv_file_name)

json_df.to_csv(csv_file_name, sep='|', index=False, encoding='utf-8')

print (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), \
       'Размер csv файла:', \
       round(os.path.getsize(csv_file_path)/(1000*1000.0),2), \
       'Мегабайт')